
In this notebook we import glitch data from the GWOSC webpage, whiten it and crop it. This data can then be used for the training of generative models.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install gwpy

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd

import os
import h5py as h5
import matplotlib.pyplot as plt
from gwpy.timeseries import TimeSeries
from os import listdir
import numpy as np

from gwosc.datasets import event_gps
from gwpy.signal import filter_design
import matplotlib.mlab as mlab
from scipy.interpolate import interp1d

Only Blip glitches identified with a confidence of 90% or higher are selected.

In [ ]:
from gwpy.table import GravitySpyTable

H1_O2 = GravitySpyTable.read('/content/drive/MyDrive/GWOSC_Data/H1_O2.csv')

H1_O2[(H1_O2["ml_label"] == "Blip")& (H1_O2["ml_confidence"] > 0.9)]

In [ ]:
# function to whiten data
def whiten(strain, interp_psd, dt):
    Nt = len(strain)
    freqs = np.fft.rfftfreq(Nt, dt)
    print(Nt,dt,freqs,len(freqs))
    freqs1 = np.linspace(0,2048.,Nt//2+1)
    print(freqs1,len(freqs1))

    # whitening: transform to freq domain, divide by asd, then transform back,
    # taking care to get normalization right.
    hf = np.fft.rfft(strain)
    white_hf = hf / (np.sqrt(interp_psd(freqs) /dt/2.))
    white_ht = np.fft.irfft(white_hf, n=Nt)
    return white_ht

#function to compute the q-plot
def qplot(data, t_center, t_delta):

    plot=data.crop(t_center-t_delta, t_center+t_delta).plot(figsize=[2, 2])
    plt.axis('off')
    ax = plot.gca()
    ax.grid(False)

    return plot

We create h5 files containing the timeries of the glitches. One needs to create the folder H1_O2_Blip_startingindex-finalindex and name the resulting zip file accordingly.

In [ ]:
df=H1_O2
#setting the sampling rate and time intervals of the signals
fs=4096
dt=1.0 /fs
NFFT = int(4*fs)

for i in range(7000,10000):
  print(i)
  gps=df['event_time'][i]
  try:
    #fetch the data corresponding to the gps time and whiten it
    data = TimeSeries.fetch_open_data(df['ifo'][i], gps-4, gps+4)
    #add a hack to discard timeseries with nan as elements
    if str(data[0])=='nan':
      pass
    else:
      Pxx, freqs=mlab.psd(data, Fs = fs, NFFT = NFFT)
      psd = interp1d(freqs, Pxx)

      whitened_data=whiten(data,psd,dt)
      #crop the signal to avoid spurious boundary effects
      whitened_data = whitened_data.crop(*whitened_data.span.contract(1))
      max_pos=whitened_data.argmax()

      #cut the signal to be in line with the gengli paper (938 points in total)
      whitened_data=whitened_data[int(max_pos-469):int(max_pos+469)]

      new_filename=df['gravityspy_id'][i]+'_whitened.h5'
      with h5.File('/content/sample_data/H1_O2_Blip_7000-10000/'+new_filename, "w") as f:
        #create the new dataset and add the useful meta-data
        dset = f.create_dataset('Blip', (len(whitened_data),), dtype='f')
        dset[...] =whitened_data
        dset.attrs['sample_rate']=4096.0
        dset.attrs['t0']=df['event_time'][i]
      #whitened_data.plot()
      #plt.show()
  except:
      pass
  #if i == stop_at:
  #        break
!zip -r /content/sample_data/H1_O2_Blip_7000-10000.zip /content/sample_data/H1_O2_Blip_7000-10000
files.download("/content/sample_data/H1_O2_Blip_7000-10000.zip")